In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [10]:
prototxtPath=os.path.sep.join([r'D:\face-mask-detector','deploy.prototxt'])
weightsPath=os.path.sep.join([r'D:\face-mask-detector','res10_300x300_ssd_iter_140000.caffemodel'])

In [11]:
prototxtPath

'D:\\face-mask-detector\\deploy.prototxt'

In [12]:
weightsPath

'D:\\face-mask-detector\\res10_300x300_ssd_iter_140000.caffemodel'

In [13]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [14]:
model=load_model(r'D:\face-mask-detector\mobilenet_v2.model')

In [22]:
image=cv2.imread(r'D:\face-mask-detector\20190827_172001.jpg')

In [23]:
image

array([[[192, 197, 195],
        [194, 199, 197],
        [197, 202, 200],
        ...,
        [204, 208, 202],
        [208, 212, 206],
        [217, 221, 215]],

       [[193, 198, 196],
        [192, 197, 195],
        [194, 199, 197],
        ...,
        [201, 205, 199],
        [211, 215, 209],
        [219, 223, 217]],

       [[193, 198, 196],
        [193, 198, 196],
        [193, 198, 196],
        ...,
        [204, 208, 202],
        [214, 218, 212],
        [219, 223, 217]],

       ...,

       [[117, 124, 127],
        [120, 127, 130],
        [123, 129, 134],
        ...,
        [ 88,  93,  91],
        [ 88,  93,  91],
        [ 88,  93,  91]],

       [[117, 124, 127],
        [120, 127, 130],
        [123, 129, 134],
        ...,
        [ 93,  98,  96],
        [ 93,  98,  96],
        [ 93,  98,  96]],

       [[117, 124, 127],
        [120, 127, 130],
        [123, 129, 134],
        ...,
        [ 99, 104, 102],
        [ 99, 104, 102],
        [ 99, 104, 102]]

In [24]:
(h,w)=image.shape[:2]
(h,w)

(1583, 1403)

In [25]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))
blob

array([[[[  89.,   90.,   86., ...,  110.,   23.,  101.],
         [  85.,   79.,   75., ...,  106.,   23.,   99.],
         [ -11.,   -8.,   -8., ...,  108.,   24.,   97.],
         ...,
         [  17.,   25.,   26., ...,  -36.,  -38.,  -39.],
         [  17.,   25.,   26., ...,  -35.,  -35.,  -35.],
         [  18.,   25.,   26., ...,  -15.,  -15.,  -17.]],

        [[  21.,   22.,   19., ...,   36.,  -50.,   32.],
         [  17.,    9.,    6., ...,   32.,  -49.,   31.],
         [ -83.,  -81.,  -81., ...,   33.,  -48.,   29.],
         ...,
         [ -49.,  -39.,  -36., ..., -112., -112., -112.],
         [ -50.,  -39.,  -36., ..., -108., -108., -108.],
         [ -48.,  -39.,  -36., ...,  -86.,  -85.,  -85.]],

        [[  73.,   74.,   71., ...,   80.,   -9.,   80.],
         [  67.,   57.,   54., ...,   77.,   -8.,   76.],
         [ -37.,  -39.,  -41., ...,   77.,   -7.,   74.],
         ...,
         [   9.,   19.,   22., ...,  -56.,  -56.,  -57.],
         [   9.,   19.,   

In [26]:
blob.shape

(1, 3, 300, 300)

In [27]:
net.setInput(blob)
detections=net.forward()
detections

array([[[[0.        , 1.        , 0.99996424, ..., 0.09913768,
          0.7198938 , 0.58546543],
         [0.        , 1.        , 0.11593315, ..., 4.015192  ,
          4.8457193 , 4.9840813 ],
         [0.        , 1.        , 0.11162671, ..., 4.015235  ,
          0.83312833, 4.987076  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [28]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()